(Turn on equation numbering first.)

In [ ]:
%%javascript
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});

This note book is about learning some theory basics on time series and their analysis. _Stationary_ time series play an important role in time series analysis. Therefore, let's start with a definition: a time series $\{y_t\}$ is _covariance stationary_ (or _weakly stationary_) if its expectation value and (auto-)covariances are time independent:
\begin{align}
\mathrm{E}[y_t] &= \mu \label{eq:stationary1} \\
\mathrm{E}[(y_t - \mu)(y_{t-j} - \mu)] &= \gamma_j \label{eq:stationary2}
\end{align}
For the rest of this notebook, we will refer to _covariance stationary_ using the short-hand (but somehow imprecise) _stationary_.

As an example consider a simple random walk model
\begin{equation}
y_t = y_{t-1} + u_t \label{eq:rw}
\end{equation}
where $u_t$ is Gaussian white noise ($u_t \sim \mathcal{N}(0,\sigma^2)$). For simplicity we define $y_0 = 0$ which simplifies \eqref{eq:rw} to 
\begin{equation}
y_t = \sum_{i=1}^t u_t. \label{eq:rw_simple}
\end{equation}

So let's start with generating an example time series for 100 time steps and plot it.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
%matplotlib inline

First let's define some helper functions.

In [ ]:
def plot_sample(y,samples= 5):
    """
    input: y       ... ensemble of timeseries as np.array with shape (timesteps,ensemble_size)
           samples ... number of individual time series to plot
    """
    plt.plot(y[:,:samples])
    plt.xlabel('$t$')
    plt.ylabel('$y_t$');

In [ ]:
timesteps = 100
# set mean and variance of the noise
mean = 0
var = 1
# generate the Gaussian white noise
noise = np.random.normal(mean,sqrt(var),size=timesteps)
# construct and plot the time series
rw = np.cumsum(noise)
plot_sample(rw.add,1)

The question now is whether time series generated by such a model are _stationary_. In order to answer this question, we need to check the conditions \eqref{eq:stationary1} and \eqref{eq:stationary2} outlined above. This can be done by generating an ensemble of time series for this model and evaluate the mean and the (auto-)covariances as function of the timestep $t$.

In [ ]:
ensemble_size = 1000
# generate an ensemble of the Gaussian white noise
noise = np.random.normal(mean,sqrt(var),size=(timesteps,ensemble_size))
# summing along the first axis yields the different time series
rw = np.cumsum(noise,axis=0)
# make an example plot for the first 5 time series of the ensemble
plt.plot(rw[:,:5])
plt.xlabel('$t$')
plt.ylabel('$y_t$');

Now that we have an ensemble of time series generated by this model, we can test the conditions for stationarity.

In [ ]:
# get the mean and covariances as function of the time
# this time we average of the second axis which corresponds to the different time series in the ensemble
means = np.mean(rw,axis=1)
# calculate variance from ensemble (correct for bias)
vars = np.var(rw,axis=1,ddof=1)
_,(ax1,ax2) = plt.subplots(nrows = 2, sharex=True,figsize=(8,8))
ax1.plot(means)
ax1.set_xlabel('$t$')
ax1.set_ylabel('$\mathrm{E}[y_t]$')
ax2.plot(vars)
ax2.set_xlabel('$t$')
ax2.set_ylabel('$\sigma^2(y_t)$');

As one can see the expected values for $y_t$ are rather stable while the variance $\sigma^2 (y_t)$ is increasing linearly with $t$. This can easily be seen from equation \eqref{eq:rw_simple} above since $$\sigma^2 (y_t) = \sum_{i=1}^t \sigma^2 (u_t) = t \sigma^2.$$
Therefore, random walk models do **not** produce stationary time series.

_Moving average processes of order $q$_ form another class of time series and are called MA(q). They are defined by

\begin{equation}
y_t = \mu + u_t + \sum_{i=1}^q \theta_i u_{t - i} = \mu + \theta^\intercal u \label{eq:maq}
\end{equation}

with $\theta^\intercal = \left(\theta_0 = 1, \theta_1 \dots \theta_q \right)$ and $u = \left(u_t, u_{t-1} \dots u_{t-q} \right)^\intercal$ where $\mu$ and $\theta_i$ are constants while $\{u_t\}$ is white noise ($\sim iid(0,\sigma^2)$). From \eqref{eq:maq} one can easily derive the expectation value and (auto-)covariances for $y_t$:

\begin{align}
\mathrm{E}[y_t] &= \mu \\
\gamma_0 &= \sigma^2 \sum_{i=0}^q \theta_i^2 \\
\gamma_j &= \begin{cases} \sigma^2 \sum_{i=0}^{q-j} \theta_i \theta_{i+j} & \text{if } 1 \le j \le q \\ 0 & \text{if } j > q\end{cases}
\end{align}

Since these are all time-independent, MA(q) models generate stationary time series.

In [ ]:
def MA_Q(mu,theta,q,var,timesteps,ensemble_size):
    """
    generate MA(q) time series
    
    input: mu            ... mean of time series
           theta         ... coefficient vector (including theta_0 = 1)
           q             ... order of MA process
           var           ... variance of white noise
           timesteps     ... number of time steps to generate the process for
           ensemble_size ... number of time series to generate
           
    returns: y     ... ensemble of MA(q) time series with shape (timesteps, ensemble_size)
             gamma ... vector (auto-)covariances up to j = q
    """
    y = np.zeros((timesteps,ensemble_size))
    # account for theta_0
    ut = np.zeros((q+1,ensemble_size))
    for t in np.arange(timesteps):
        ut = np.roll(ut,1,axis=0)
        ut[0,:] = np.random.normal(0,sqrt(var),size=(1,ensemble_size))
        y[t,:] = mu + np.dot(theta,ut)
    
    # covariance and q autocovariances for lags 1...q
    covars = np.zeros(q+1)
    covars[0] = np.sum(theta**2)
    for j in np.arange(1,q+1):
        covars[j] = np.sum([theta[a] * theta[a-j] for a in np.arange(j,q+1)])
    covars *= var
    return y,covars

In [ ]:
# set coefficients (up to 6) and add theta_0 = 1 at the beginning
thetas = np.random.rand(6)
thetas = np.insert(thetas,0,1)

# get an ensemble of MA(6) time series with the theortical values for the mean expectation and autocovariances
ma6, gamma = MA_Q(1.7,thetas,6,1,timesteps,ensemble_size)
# make an example plot for the first 5 time series of the ensemble
plt.plot(ma6[:,:5])
plt.xlabel('$t$')
plt.ylabel('$y_t$');

In [ ]:
# get the mean as function of the time
mu = 1.7
means = np.mean(ma6,axis=1)
# calculate auto-covariance for certain lags from ensemble
lags = np.arange(7)
covars = np.vstack([np.hstack([np.cov(ma6[t,:],ma6[t-j,:],ddof=1)[0,1] for t in np.arange(timesteps)]) for j in lags])
_,axes = plt.subplots(ncols = 2, nrows = 4, sharex=True,figsize=(12,9))
axes = axes.flatten()
axes[0].plot(means)
axes[0].hlines(mu,0,100,'r')
axes[0].set_xlabel('$t$')
axes[0].set_ylabel('$\mathrm{E}[y_t]$')
for i in np.arange(1,len(lags)+1):
    axes[i].plot(covars[i-1,:])
    axes[i].hlines(gamma[i-1],0,100,'r')
    axes[i].set_xlabel('$t$')
    axes[i].set_ylabel('$\gamma_%d$' % lags[i-1]);

The plots nicely illustrate that MA(q) processes are covariance-stationary.

_Autoregressive processes of order p_ are yet another class of time series models denoted AR(p). They are defined by

\begin{equation}
y_t = c + \sum_{i=1}^p \phi_i y_{t-i} + u_t \label{eq:ar}
\end{equation}

with $c$, $\phi_i$ being constants and $\{u_t\}$ being again white noise ($\sim iid(0,\sigma^2)$). A AR(1) process can be written as

\begin{equation}
y_t = c + \phi y_{t-1} + u_t = c \sum_{i=0}^{t-1} \phi^i + \sum_{i=0}^{t-1} \phi^i u_{t-i} + \phi^t y_0 \label{eq:ar1}
\end{equation}

where the right-hand side is the MA(t) representation of an AR(1) process. Assuming $|\phi| < 1$ the effect of the initial condition will decay with time and

\begin{equation}
\lim_{t \to \infty} \phi^t y_0 \to 0 .
\end{equation}

In this case, the time series can be assumed to have started long ago in the past and \eqref{eq:ar1} simplifies to (using $\sum_{i=0}^\infty \phi^i = \left( 1 - \phi \right)^{-1}$)

\begin{equation}
y_t = \frac{c}{1 - \phi} + \sum_{i=0}^\infty \phi^i u_{t-i} .
\end{equation}

The special case $c= 0$ and $\phi=1$ is called random walk and was already discussed above.

In [ ]:
def ar_p(c,phi,var,p=3):
    """
    generate AR(p) time series
    
    returns: y     ... ensemble of time series
             mu    ... expectation value
             gamma ... (auto-)covariances up to j = q + 1
    """
    y = np.zeros((timesteps,ensemble_size))
    tmp = np.zeros((p,ensemble_size))
    for t in np.arange(timesteps):
        y[t,:] = c + np.dot(phi,tmp) + np.random.normal(0,sqrt(var),size=(1,ensemble_size))
        tmp = np.roll(tmp,1,axis=0)
        tmp[0,:] = y[t,:]
    
    return y

In [ ]:
# get an ensemble of AR(6) time series with the theortical values for the mean expectation and autocovariances
p = 6
phis = np.random.normal(0,0.3,6)
print(phis)
ar6 = ar_p(0.3,phis,0.7,p)
# make an example plot for the first 5 time series of the ensemble
plt.plot(ar6[:,:5])
plt.xlabel('$t$')
plt.ylabel('$y_t$');

In [ ]:
# get the mean as function of the time
means = np.mean(ar6,axis=1)
# calculate auto-covariance for certain lags from ensemble
lags = np.arange(7)
covars = np.vstack([np.hstack([np.cov(ar6[t,:],ar6[t-j,:],ddof=1)[0,1] for t in np.arange(timesteps)]) for j in lags])
_,axes = plt.subplots(ncols = 2, nrows = 4, sharex=True,figsize=(12,9))
axes = axes.flatten()
axes[0].plot(means)
axes[0].hlines(mu,0,100,'r')
axes[0].set_xlabel('$t$')
axes[0].set_ylabel('$\mathrm{E}[y_t]$')
for i in np.arange(1,len(lags)+1):
    axes[i].plot(covars[i-1,:])
    axes[i].hlines(gamma[i-1],0,100,'r')
    axes[i].set_xlabel('$t$')
    axes[i].set_ylabel('$\gamma_%d$' % lags[i-1]);

In [ ]:
from statsmodels.tsa.stattools import acovf
print(acovf(ma3[10:,0],unbiased=True)[1])
print(acovf(ma3[10:,0] - 10)[1])
print(np.cov(ma3[10:-1,0],ma3[11:,0],ddof=0))
print((np.correlate(ma3[10:,0]-100,ma3[10:,0]-100,'full')/90)[89])

In [ ]:
?np.arange